In [1]:
%load_ext autoreload
%autoreload 2

import time
import threading
import logging
from typing import Optional

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander
from cflib.utils.multiranger import Multiranger

from cfpilot.controller import CrazyflieController
from cfpilot.controller import CrazyflieController


# Setup logging to see output in notebook
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("✅ Imports complete")

✅ Imports complete


In [19]:
controller = CrazyflieController(enable_plotting=False)
config = controller.config
# Initialize drivers
cflib.crtp.init_drivers()

# Connect to Crazyflie (will auto-detect or use config URI)
uri = config['connection']['uri']
controller.connect(uri)

# Wait for connection
if controller.wait_for_connection(timeout=10.0):
    logger.info("✅ Connected successfully!")
else:
    logger.error("❌ Connection failed!")
    raise Exception("Could not connect to Crazyflie")

2025-10-05 18:07:46,576 - cfpilot.detection - INFO - Landing pad detector initialized
2025-10-05 18:07:46,577 - cfpilot.controller - INFO - Crazyflie Controller initialized (Async)
2025-10-05 18:07:46,578 - cfpilot.controller - INFO - Connecting to radio://0/88/2M/E7E7E7E7F0
2025-10-05 18:07:46,579 - cflib.crazyflie - INFO - Callback->Connection initialized[radio://0/88/2M/E7E7E7E7F0]


2025-10-05 18:07:46,863 - cflib.crazyflie - INFO - We are connected[radio://0/88/2M/E7E7E7E7F0], request connection setup
2025-10-05 18:07:46,864 - cflib.crazyflie.platformservice - INFO - Request _request_protocol_version()
2025-10-05 18:07:46,866 - cflib.crazyflie - INFO - Callback->Connected to [radio://0/88/2M/E7E7E7E7F0]
2025-10-05 18:07:46,868 - cflib.crazyflie.log - WARNING - Error no LogEntry to handle id=1
2025-10-05 18:07:46,869 - cflib.crazyflie.log - WARNING - Error no LogEntry to handle id=2
2025-10-05 18:07:46,870 - cflib.crazyflie.platformservice - INFO - _crt_service_callback
2025-10-05 18:07:46,870 - cflib.crazyflie.platformservice - INFO - Request protocol version
2025-10-05 18:07:46,873 - cflib.crazyflie.platformservice - INFO - _platform_callback
2025-10-05 18:07:46,874 - cflib.crazyflie.platformservice - INFO - Protocol version (platform): 10
2025-10-05 18:07:46,888 - cflib.crazyflie.toc - INFO - TOC for port [5] found in cache
2025-10-05 18:07:46,888 - cflib.crazy

In [21]:
# Cell 4: Simple Test Flight

try:
    # Setup position commander
    pc = PositionHlCommander(
        controller.cf,
        default_velocity=0.3,
        default_height=0.5
    )
    mc = MotionCommander(
        controller.cf,
        default_height=0.5
    )
    
    # Start multiranger for obstacle detection
    multiranger = Multiranger(controller.cf)
    multiranger.start()
    logger.info("✅ Multiranger started")
    
    # Takeoff
    logger.info("🚁 Taking off...")
    height = 0.5
    mc.take_off(height=height)
    # pc.take_off(height=height)
    time.sleep(1)

    latest = controller.get_latest_data()
    x = latest.get('stateEstimate.x', 0)
    y = latest.get('stateEstimate.y', 0)
    z = latest.get('stateEstimate.z', 0)

    target_x = 2.0
    mc.forward(2.0)


    latest = controller.get_latest_data()
    x = latest.get('stateEstimate.x', 0)
    y = latest.get('stateEstimate.y', 0)
    z = latest.get('stateEstimate.z', 0)

    deviate = x - target_x

    while abs(deviate) > 0.05:
        logger.info(f"📍 Current position: ({x:.2f}, {y:.2f}, {z:.2f}), Deviate: {deviate:.2f}")
        if deviate > 0:
            mc.back(min(0.05, abs(deviate)))
            logger.info("Moving back")
        else:
            mc.forward(min(0.05, abs(deviate)))
            logger.info("Moving forward")

        time.sleep(0.1)
        latest = controller.get_latest_data()
        x = latest.get('stateEstimate.x', 0)
        y = latest.get('stateEstimate.y', 0)
        z = latest.get('stateEstimate.z', 0)
        deviate = x - target_x




    time.sleep(1)
    mc.land()
    # Land
    logger.info("🛬 Landing...")
    # pc.land()

    current_time = time.time()
    while time.time() - current_time < 3:
        latest = controller.get_latest_data()
    
        logger.info(f"📍 Current position: ({latest.get('stateEstimate.x', 0):.2f}, "
                   f"{latest.get('stateEstimate.y', 0):.2f}, {latest.get('stateEstimate.z', 0):.2f})")

        time.sleep(0.1) 
    
    # Stop multiranger
    multiranger.stop()
    controller.save_flight_data()
    logger.info("✅ Flight complete!")
    
except Exception as e:
    logger.error(f"❌ Error during flight: {e}")
    controller.emergency_shutdown()
    raise

2025-10-05 18:13:41,845 - __main__ - INFO - ✅ Multiranger started
2025-10-05 18:13:41,846 - __main__ - INFO - 🚁 Taking off...
2025-10-05 18:13:41,852 - cflib.crazyflie.log - INFO - Have successfully started logging for id=4


2025-10-05 18:13:57,447 - __main__ - INFO - 📍 Current position: (2.07, -0.03, 0.51), Deviate: 0.07
2025-10-05 18:13:57,698 - __main__ - INFO - Moving back
2025-10-05 18:13:57,798 - __main__ - INFO - 📍 Current position: (2.09, -0.03, 0.51), Deviate: 0.09
2025-10-05 18:13:58,049 - __main__ - INFO - Moving back
2025-10-05 18:14:01,651 - __main__ - INFO - 🛬 Landing...
2025-10-05 18:14:01,652 - __main__ - INFO - 📍 Current position: (2.05, -0.03, 0.09)
2025-10-05 18:14:01,752 - __main__ - INFO - 📍 Current position: (2.04, -0.02, 0.06)
2025-10-05 18:14:01,853 - __main__ - INFO - 📍 Current position: (2.04, -0.02, 0.01)
2025-10-05 18:14:01,954 - __main__ - INFO - 📍 Current position: (2.04, -0.02, 0.02)
2025-10-05 18:14:02,055 - __main__ - INFO - 📍 Current position: (2.04, -0.00, 0.03)
2025-10-05 18:14:02,155 - __main__ - INFO - 📍 Current position: (2.04, 0.01, 0.03)
2025-10-05 18:14:02,256 - __main__ - INFO - 📍 Current position: (2.04, 0.01, 0.03)
2025-10-05 18:14:02,357 - __main__ - INFO - 📍 C

In [10]:
latest

{}